In [7]:
import json
import pdfplumber
import docx
import gradio as gr
import requests
from datetime import datetime

# Constants for Llama API
OLLAMA_API = "http://localhost:11434/api/chat"  # Llama API endpoint
MODEL = "llama3.2"  # Llama model version

# Extract structured information from resume using Llama
EXTRACT_FEATURES_FROM_RESUME = """
Extract the following structured information from the resume:
{
    "personal": {
        "name": "",
        "email": "",
        "phone": "",
        "linkedin": "",
        "github": "",
        "research_publications": []
    },
    "skills": [
        {
            "name": "",
            "type": "technical" or "soft",
            "proficiency": ""
        }
    ],
    "education": [
        {
            "degree": "",
            "institution": "",
            "year": ""
        }
    ],
    "work_experience": [
        {
            "company": "",
            "role": "",
            "duration": ""
        }
    ],
    "overall_experience": 0
}
"""

# Function to extract text from the resume (PDF or DOCX)
def extract_text_from_resume(file_path):
    text = ""
    if file_path.endswith(".pdf"):
        with pdfplumber.open(file_path) as pdf:
            text = "\n".join([page.extract_text() for page in pdf.pages if page.extract_text()])
    elif file_path.endswith(".docx"):
        doc = docx.Document(file_path)
        text = "\n".join([para.text for para in doc.paragraphs])
    return text.strip()

# Function to send the extracted text to Llama for information extraction
def extract_data_from_llama(text):
    prompt = EXTRACT_FEATURES_FROM_RESUME  # Prompt for Llama API
    payload = {
        "model": MODEL,
        "input": text,
        "prompt": prompt
    }

    # Sending the request to Llama API
    response = requests.post(OLLAMA_API, json=payload)

    if response.status_code == 200:
        return response.json()  # Return structured information from Llama
    else:
        return {"error": "Failed to extract data from resume. Check the API."}

# Function to calculate match score between resume and job role
def calculate_match(extracted_data, job_role):
    job_requirements = {
        "Software Engineer": {
            "technical_skills": ["Python", "Java", "Cloud Computing", "Docker", "Kubernetes", "CI/CD", "Git", "SQL", "REST APIs"],
            "soft_skills": ["Problem Solving", "Teamwork", "Communication", "Agile Methodology", "Critical Thinking"],
            "required_info": ["email", "phone", "linkedin"],
            "min_experience": 2
        },
        "Data Scientist": {
            "technical_skills": ["Machine Learning", "Deep Learning", "Data Visualization", "Python", "R", "SQL", "Pandas", "NumPy", "Scikit-learn", "TensorFlow", "Keras"],
            "soft_skills": ["Analytical Thinking", "Communication", "Storytelling", "Problem Solving", "Research Skills"],
            "required_info": ["email", "phone", "github"],
            "min_experience": 1.5
        },
        "AI Engineer": {
            "technical_skills": ["Neural Networks", "NLP", "Computer Vision", "TensorFlow", "PyTorch", "Deep Learning", "Python", "Machine Learning", "Keras"],
            "soft_skills": ["Research Skills", "Innovation", "Problem Solving", "Analytical Thinking", "Creativity"],
            "required_info": ["email", "phone", "research_publications"],
            "min_experience": 2
        },
        "Cybersecurity Analyst": {
            "technical_skills": ["Ethical Hacking", "Threat Detection", "Network Security", "Penetration Testing", "SIEM", "Firewall Configuration", "Vulnerability Assessment", "Security Protocols"],
            "soft_skills": ["Attention to Detail", "Analytical Thinking", "Risk Assessment", "Problem Solving", "Communication"],
            "required_info": ["email", "phone"],
            "min_experience": 1
        }
    }

    # Get job role requirements
    requirements = job_requirements.get(job_role, {})

    # Extract candidate skills
    candidate_skills = {
        "technical_skills": [skill["name"] for skill in extracted_data.get("skills", []) if skill.get("type") == "technical"],
        "soft_skills": [skill["name"] for skill in extracted_data.get("skills", []) if skill.get("type") == "soft"]
    }

    # Match skills
    matched_technical = [skill for skill in requirements.get("technical_skills", []) if skill in candidate_skills["technical_skills"]]
    missing_technical = [skill for skill in requirements.get("technical_skills", []) if skill not in candidate_skills["technical_skills"]]
    matched_soft = [skill for skill in requirements.get("soft_skills", []) if skill in candidate_skills["soft_skills"]]
    missing_soft = [skill for skill in requirements.get("soft_skills", []) if skill not in candidate_skills["soft_skills"]]

    # Missing personal info
    missing_info = [info for info in requirements.get("required_info", []) if not extracted_data.get("personal", {}).get(info)]

    # Experience check
    experience = extracted_data.get("overall_experience", 0)
    meets_experience = experience >= requirements.get("min_experience", 0)

    # Match Score Calculation
    technical_weight = 0.4
    soft_skills_weight = 0.2
    experience_weight = 0.2
    info_weight = 0.2

    technical_score = (len(matched_technical) / max(len(requirements.get("technical_skills", [])), 1)) * 100 * technical_weight
    soft_skills_score = (len(matched_soft) / max(len(requirements.get("soft_skills", [])), 1)) * 100 * soft_skills_weight
    experience_score = (experience / max(requirements.get("min_experience", 1), 1)) * 100 * experience_weight
    info_score = ((len(requirements.get("required_info", [])) - len(missing_info)) / max(len(requirements.get("required_info", [])), 1)) * 100 * info_weight

    overall_score = round(technical_score + soft_skills_score + experience_score + info_score, 2)

    # Return the detailed result
    return {
        "Job Role": job_role,
        "Overall Match Score": f"{overall_score}%",
        "Matched Technical Skills": matched_technical,
        "Missing Technical Skills": missing_technical,
        "Matched Soft Skills": matched_soft,
        "Missing Soft Skills": missing_soft,
        "Experience": f"{experience} years (Required: {requirements.get('min_experience', 0)} years)",
        "Missing Personal Info": missing_info
    }

# Function to process the resume and compute match
def process_resume(file, job_role):
    if not file:
        return "Please upload a resume."

    file_path = file.name
    text = extract_text_from_resume(file_path)

    if not text:
        return "Could not extract text from the resume."

    # Extract data from the resume using Llama
    extracted_data = extract_data_from_llama(text)
    
    if "error" in extracted_data:
        return extracted_data["error"]

    # Calculate match score for the job role
    return calculate_match(extracted_data, job_role)

# Gradio Interface
with gr.Blocks() as demo:
    gr.Markdown("# Resume Matching System")
    file_input = gr.File(label="Upload Resume (PDF/DOCX)")
    job_role_input = gr.Dropdown(choices=["Software Engineer", "Data Scientist", "AI Engineer", "Cybersecurity Analyst"], label="Select Job Role")
    submit_button = gr.Button("Analyze Resume")
    output = gr.JSON(label="Match Results")

    submit_button.click(process_resume, inputs=[file_input, job_role_input], outputs=output)

# Launch the Gradio app
demo.launch()


INFO:httpx:HTTP Request: GET http://127.0.0.1:7900/gradio_api/startup-events "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: HEAD http://127.0.0.1:7900/ "HTTP/1.1 200 OK"


* Running on local URL:  http://127.0.0.1:7900

To create a public link, set `share=True` in `launch()`.


INFO:httpx:HTTP Request: GET https://api.gradio.app/pkg-version "HTTP/1.1 200 OK"


In [3]:
import json
import re
import pdfplumber
import docx
import gradio as gr
from openai import OpenAI
from datetime import datetime

# Constants
OLLAMA_API = "http://localhost:11434/api/chat"
MODEL = "llama3.2"

# Prompt for feature extraction
EXTRACT_FEATURES_FROM_RESUME = """
Extract the following structured information from the resume:
{
    "personal": {
        "name": "",
        "email": "",
        "phone": "",
        "linkedin": "",
        "github": "",
        "research_publications": []
    },
    "skills": [
        {
            "name": "",
            "type": "technical" or "soft",
            "proficiency": ""
        }
    ],
    "education": [
        {
            "degree": "",
            "institution": "",
            "year": ""
        }
    ],
    "work_experience": [
        {
            "company": "",
            "role": "",
            "duration": ""
        }
    ],
    "overall_experience": 0  # Total years of experience
}

Rules:
1. Ensure all keys are present, use empty strings/lists if not found
2. Categorize skills as 'technical' or 'soft'
3. Calculate overall experience from work experience entries
4. Extract all relevant details from the resume
5. Provide response in strict JSON format
"""

# Initialize OpenAI client for Ollama
ollama_via_openai = OpenAI(base_url="http://localhost:11434/v1", api_key="ollama")

# Function to extract text from PDF or DOCX
def extract_text_from_resume(file_path):
    text = ""
    if file_path.endswith(".pdf"):
        with pdfplumber.open(file_path) as pdf:
            for page in pdf.pages:
                text += page.extract_text() + "\n"
    elif file_path.endswith(".docx"):
        doc = docx.Document(file_path)
        for para in doc.paragraphs:
            text += para.text + "\n"
    else:
        raise ValueError("Unsupported file format. Use PDF or DOCX.")
    return text.strip()

# Function to calculate years of experience from a work duration string
def calculate_experience_years(duration):
    try:
        months_in_year = 12
        # Split by space and look for year/month pattern
        if "year" in duration.lower():
            years = re.findall(r"(\d+)\s*year", duration.lower())
            if years:
                return float(years[0])
        if "month" in duration.lower():
            months = re.findall(r"(\d+)\s*month", duration.lower())
            if months:
                return float(months[0]) / months_in_year
    except Exception as e:
        print(f"Error in calculating experience: {e}")
    return 0

# Enhanced Function to calculate match score & provide detailed insights
def calculate_match(extracted_data, job_description):
    # Comprehensive skill requirements for different job roles
    required_skills = {
        "Software Engineer": {
            "technical": ["Python", "Java", "Cloud Computing", "Docker", "Kubernetes", "CI/CD", "Git"],
            "soft_skills": ["Problem Solving", "Teamwork", "Communication"],
            "required_info": ["email", "phone", "linkedin"]
        },
        "Data Scientist": {
            "technical": ["Machine Learning", "Deep Learning", "Data Visualization", "Python", "R", "SQL", "Pandas", "NumPy", "Scikit-learn"],
            "soft_skills": ["Analytical Thinking", "Communication", "Storytelling"],
            "required_info": ["email", "phone", "github"]
        },
        "AI Engineer": {
            "technical": ["Neural Networks", "NLP", "Computer Vision", "TensorFlow", "PyTorch", "Deep Learning", "Python"],
            "soft_skills": ["Research Skills", "Innovation", "Problem Solving"],
            "required_info": ["email", "phone", "research_publications"]
        },
        "Cybersecurity Analyst": {
            "technical": ["Ethical Hacking", "Threat Detection", "Network Security", "Penetration Testing", "SIEM", "Firewall Configuration"],
            "soft_skills": ["Attention to Detail", "Analytical Thinking", "Risk Assessment"],
            "required_info": ["email", "phone", "security_certifications"]
        }
    }
    
    # Get job role from job description
    job_role = job_description.split("with ")[-1].split(" experience")[0]
    job_skills = required_skills.get(job_role, {})
    
    # Extract skills from resume
    extracted_skills = {
        "technical_skills": [skill["name"] for skill in extracted_data.get("skills", []) if skill.get("type") == "technical"],
        "soft_skills": [skill["name"] for skill in extracted_data.get("skills", []) if skill.get("type") == "soft"]
    }
    
    # Skill Matching
    matched_technical_skills = [skill for skill in job_skills.get("technical", []) if skill in extracted_skills["technical_skills"]]
    missing_technical_skills = [skill for skill in job_skills.get("technical", []) if skill not in extracted_skills["technical_skills"]]
    
    matched_soft_skills = [skill for skill in job_skills.get("soft_skills", []) if skill in extracted_skills["soft_skills"]]
    missing_soft_skills = [skill for skill in job_skills.get("soft_skills", []) if skill not in extracted_skills["soft_skills"]]
    
    # Missing Personal Information
    missing_personal_info = [info for info in job_skills.get("required_info", []) 
                              if not extracted_data.get("personal", {}).get(info)]
    
    # Match Score Calculation
    technical_skill_weight = 0.4  # 40% importance for technical skills
    soft_skill_weight = 0.2       # 20% importance for soft skills
    experience_weight = 0.2       # 20% importance for experience
    info_completeness_weight = 0.2  # 20% importance for personal info completeness
    
    # Technical Skills Score
    technical_skill_score = min(1.0, len(matched_technical_skills) / len(job_skills.get("technical", [1]))) * technical_skill_weight * 100
    
    # Soft Skills Score
    soft_skill_score = min(1.0, len(matched_soft_skills) / len(job_skills.get("soft_skills", [1]))) * soft_skill_weight * 100
    
    # Experience Score
    experience_score = min(50, extracted_data.get("overall_experience", 0) * 10) * experience_weight
    
    # Information Completeness Score
    info_completeness_score = (1 - (len(missing_personal_info) / len(job_skills.get("required_info", [1])))) * info_completeness_weight * 100
    
    # Total Match Score
    match_score = round(technical_skill_score + soft_skill_score + experience_score + info_completeness_score, 2)
    
    # Detailed Match Analysis
    match_breakdown = {
        "technical_skill_score": round(technical_skill_score, 2),
        "soft_skill_score": round(soft_skill_score, 2),
        "experience_score": round(experience_score, 2),
        "info_completeness_score": round(info_completeness_score, 2)
    }
    
    # Update extracted data with match details
    extracted_data.update({
        "match_score": match_score,
        "match_breakdown": match_breakdown,
        "skills_analysis": {
            "matched_technical_skills": matched_technical_skills,
            "missing_technical_skills": missing_technical_skills,
            "matched_soft_skills": matched_soft_skills,
            "missing_soft_skills": missing_soft_skills
        },
        "missing_personal_info": missing_personal_info
    })
    
    return extracted_data

# Function to query Llama 3.2
def query_llama_api(resume_text, job_description):
    try:
        prompt = f"Resume Text: {resume_text}\nJob Description: {job_description}\n\n{EXTRACT_FEATURES_FROM_RESUME}\nEnsure the output is strictly in JSON format without any additional text."

        response = ollama_via_openai.completions.create(
            model=MODEL,
            prompt=prompt,
            max_tokens=1000,  # Increase to capture full JSON
            temperature=0.7,
        )

        raw_output = response.choices[0].text.strip()
        print("Raw API Response:", raw_output)  # Debugging step

        match = re.search(r"\{.*\}", raw_output, re.DOTALL)
        if match:
            parsed_data = json.loads(match.group(0))
            
            # Calculate overall experience
            if parsed_data.get("work_experience"):
                experience_years = sum(
                    calculate_experience_years(exp.get("duration", "0"))
                    for exp in parsed_data["work_experience"] 
                    if exp.get("duration")
                )
                parsed_data["overall_experience"] = experience_years
            else:
                parsed_data["overall_experience"] = 0
            
            return parsed_data
        else:
            return {"error": "Invalid JSON extracted from response"}

    except json.JSONDecodeError:
        return {"error": "Failed to parse JSON from API response"}
    except Exception as e:
        return {"error": f"API request failed: {str(e)}"}

# Function to process the resume
def process_resume(file, job_role):
    try:
        file_path = file.name
        resume_text = extract_text_from_resume(file_path)

        job_descriptions = {
            "Software Engineer": "Looking for a software engineer with Python, Java, and cloud experience.",
            "Data Scientist": "Seeking a data scientist proficient in machine learning, deep learning, and data visualization.",
            "AI Engineer": "Hiring an AI Engineer with expertise in neural networks, NLP, and computer vision.",
            "Cybersecurity Analyst": "Looking for a cybersecurity analyst skilled in ethical hacking and threat detection."
        }

        job_description = job_descriptions.get(job_role, "No JD available.")
        extracted_data = query_llama_api(resume_text, job_description)
        
        # Only calculate match if extraction was successful
        if not extracted_data.get("error"):
            matched_data = calculate_match(extracted_data, job_description)
            return json.dumps(matched_data, indent=4)
        else:
            return json.dumps(extracted_data, indent=4)
    
    except Exception as e:
        return f"Error: {str(e)}"

# Gradio Interface
iface = gr.Interface(
    fn=process_resume,
    inputs=[
        gr.File(label="Upload Resume (PDF/DOCX)"),
        gr.Dropdown(
            choices=["Software Engineer", "Data Scientist", "AI Engineer", "Cybersecurity Analyst"],
            label="Select Job Role"
        )
    ],
    outputs=gr.JSON(label="Extracted Resume Data & Match Analysis"),
    title="Resume Analyzer & JD Matcher",
    description="Upload a resume and select a job role to extract structured information, match percentage, and missing information.",
)

# Launch the Gradio app
if __name__ == "__main__":
    iface.launch()


* Running on local URL:  http://127.0.0.1:7862

To create a public link, set `share=True` in `launch()`.


Raw API Response: {
    "personal": {
        "name": "",
        "email": "suryavishal2002@gmail.com",
        "phone": "+91 8500031155",
        "linkedin": "https://www.linkedin.com/in/surya-vishal-ganti/",
        "github": "",
        "research_publications": []
    },
    "skills": [
        {
            "name": "Python, SQL, Pandas, NumPy, Scikit-Learn, Matplotlib, OpenCV, TensorFlow, Machine Learning, Deep Learning",
            "type": "technical"
        },
        {
            "name": "Communication, Presentation, Teamwork, Problem-Solving, Time Management",
            "type": "soft"
        }
    ],
    "education": [
        {
            "degree": "B. Tech",
            "institution": "Anil Neerukonda Institute of Technology and Sciences(ANITS) Tagarapuvalasa, Vizag",
            "year": "November 2021 - April 2024"
        },
        {
            "degree": "Diploma in Electrical and Electronics Engineering",
            "institution": "Government Polytechnic Srikakul

In [5]:
import json
import re
import orjson
import pdfplumber
import docx
import gradio as gr
from openai import OpenAI
from datetime import datetime
from collections import defaultdict

# Constants
OLLAMA_API = "http://localhost:11434/api/chat"
MODEL = "llama3.2"

# Prompt for feature extraction
EXTRACT_FEATURES_FROM_RESUME = """
Extract the following structured information from the resume:
{
    "personal": {
        "name": "",
        "email": "",
        "phone": "",
        "linkedin": "",
        "github": "",
        "research_publications": []
    },
    "skills": [
        {
            "name": "",
            "type": "technical" or "soft",
            "proficiency": ""
        }
    ],
    "education": [
        {
            "degree": "",
            "institution": "",
            "year": ""
        }
    ],
    "work_experience": [
        {
            "company": "",
            "role": "",
            "duration": ""
        }
    ],
    "overall_experience": 0  # Total years of experience
}

Rules:
1. Ensure all keys are present, use empty strings/lists if not found
2. Categorize skills as 'technical' or 'soft'
3. Calculate overall experience from work experience entries
4. Extract all relevant details from the resume
5. Provide response in strict JSON format
"""

# Initialize OpenAI client for Ollama
ollama_via_openai = OpenAI(base_url="http://localhost:11434/v1", api_key="ollama")

# Function to extract text from PDF or DOCX
def extract_text_from_resume(file_path):
    text = ""
    if file_path.endswith(".pdf"):
        with pdfplumber.open(file_path) as pdf:
            for page in pdf.pages:
                text += page.extract_text() + "\n"
    elif file_path.endswith(".docx"):
        doc = docx.Document(file_path)
        for para in doc.paragraphs:
            text += para.text + "\n"
    else:
        raise ValueError("Unsupported file format. Use PDF or DOCX.")
    return text.strip()

# Function to calculate years of experience from a work duration string
def calculate_experience_years(duration):
    try:
        months_in_year = 12
        if "year" in duration.lower():
            years = re.findall(r"(\d+)\s*year", duration.lower())
            if years:
                return float(years[0])
        if "month" in duration.lower():
            months = re.findall(r"(\d+)\s*month", duration.lower())
            if months:
                return float(months[0]) / months_in_year
    except Exception as e:
        print(f"Error in calculating experience: {e}")
    return 0

# Function to query Llama for feature extraction
def query_llama_api(resume_text, job_description):
    try:
        prompt = f"Resume Text: {resume_text}\nJob Description: {job_description}\n\n{EXTRACT_FEATURES_FROM_RESUME}\nEnsure the output is strictly in JSON format without any additional text."

        response = ollama_via_openai.completions.create(
            model=MODEL,
            prompt=prompt,
            max_tokens=1000,  # Increase to capture full JSON
            temperature=0.7,
        )

        raw_output = response.choices[0].text.strip()
        print("Raw API Response:", raw_output)  # Debugging step

        # Check if the response contains valid JSON using a try-except block
        try:
            # Using orjson for more efficient JSON parsing
            parsed_data = orjson.loads(raw_output)
        except orjson.JSONDecodeError:
            return {"error": "Failed to parse JSON from API response"}
        
        # Calculate overall experience
        if parsed_data.get("work_experience"):
            experience_years = sum(
                calculate_experience_years(exp.get("duration", "0"))
                for exp in parsed_data["work_experience"] 
                if exp.get("duration")
            )
            parsed_data["overall_experience"] = experience_years
        else:
            parsed_data["overall_experience"] = 0
        
        return parsed_data

    except Exception as e:
        return {"error": f"API request failed: {str(e)}"}

# Function to match resume data with job description
def match_resume_with_job_description(extracted_data, job_description):
    job_skills = extract_job_skills(job_description)

    matched_skills = match_skills(extracted_data, job_skills)

    # Calculate the match score based on matched and missing skills
    match_score = calculate_match_score(matched_skills, extracted_data, job_description)

    return {
        "match_score": match_score,
        "skills_analysis": matched_skills,
        "personal_info": check_missing_personal_info(extracted_data, job_description)
    }

# Extract relevant job skills from the job description using simple NLP
def extract_job_skills(job_description):
    technical_keywords = ["Python", "Java", "Cloud Computing", "Docker", "Kubernetes", "CI/CD", "Git", "Machine Learning", "Deep Learning", "Data Visualization", "SQL"]
    soft_skills_keywords = ["Problem Solving", "Teamwork", "Communication", "Analytical Thinking", "Innovation"]

    technical_skills = [skill for skill in technical_keywords if skill.lower() in job_description.lower()]
    soft_skills = [skill for skill in soft_skills_keywords if skill.lower() in job_description.lower()]

    return {"technical": technical_skills, "soft_skills": soft_skills}

# Match extracted resume skills with job description skills
def match_skills(extracted_data, job_skills):
    matched_technical = [skill for skill in job_skills["technical"] if any(resume_skill["name"] == skill for resume_skill in extracted_data.get("skills", []))]
    matched_soft = [skill for skill in job_skills["soft_skills"] if any(resume_skill["name"] == skill for resume_skill in extracted_data.get("skills", []))]

    return {
        "matched_technical_skills": matched_technical,
        "matched_soft_skills": matched_soft,
        "missing_technical_skills": [skill for skill in job_skills["technical"] if skill not in matched_technical],
        "missing_soft_skills": [skill for skill in job_skills["soft_skills"] if skill not in matched_soft],
    }

# Calculate match score
def calculate_match_score(matched_skills, extracted_data, job_description):
    technical_score = len(matched_skills["matched_technical_skills"]) / len(matched_skills["matched_technical_skills"] + matched_skills["missing_technical_skills"]) * 50
    soft_score = len(matched_skills["matched_soft_skills"]) / len(matched_skills["matched_soft_skills"] + matched_skills["missing_soft_skills"]) * 30
    experience_score = min(50, extracted_data.get("overall_experience", 0) * 10)
    info_score = 20 if all(extracted_data.get("personal", {}).get(field) for field in ["email", "phone", "linkedin"]) else 0

    return round(technical_score + soft_score + experience_score + info_score, 2)

# Check if required personal info is missing
def check_missing_personal_info(extracted_data, job_description):
    required_info = ["email", "phone", "linkedin", "github", "research_publications"]
    missing_info = [field for field in required_info if not extracted_data.get("personal", {}).get(field)]

    return missing_info

# Function to process the resume and job role
def process_resume(file, job_role):
    try:
        file_path = file.name
        resume_text = extract_text_from_resume(file_path)

        extracted_data = query_llama_api(resume_text)
        
        if "error" in extracted_data:
            return json.dumps(extracted_data, indent=4)

        job_descriptions = {
            "Software Engineer": "Looking for a software engineer with Python, Java, and cloud experience.",
            "Data Scientist": "Seeking a data scientist proficient in machine learning, deep learning, and data visualization.",
            "AI Engineer": "Hiring an AI Engineer with expertise in neural networks, NLP, and computer vision.",
            "Cybersecurity Analyst": "Looking for a cybersecurity analyst skilled in ethical hacking and threat detection."
        }

        job_description = job_descriptions.get(job_role, "No JD available.")
        match_data = match_resume_with_job_description(extracted_data, job_description)
        
        return json.dumps(match_data, indent=4)
    
    except Exception as e:
        return f"Error: {str(e)}"

# Gradio Interface
iface = gr.Interface(
    fn=process_resume,
    inputs=[
        gr.File(label="Upload Resume (PDF/DOCX)"),
        gr.Dropdown(
            choices=["Software Engineer", "Data Scientist", "AI Engineer", "Cybersecurity Analyst"],
            label="Select Job Role"
        )
    ],
    outputs=gr.JSON(label="Extracted Resume Data & Match Analysis"),
    title="Resume Analyzer & JD Matcher",
    description="Upload a resume and select a job role to extract structured information, match percentage, and missing information.",
)

# Launch the Gradio app
if __name__ == "__main__":
    iface.launch()


* Running on local URL:  http://127.0.0.1:7864

To create a public link, set `share=True` in `launch()`.


Traceback (most recent call last):
  File "/Users/nitish/opt/anaconda3/envs/llms/lib/python3.11/site-packages/gradio/queueing.py", line 625, in process_events
    response = await route_utils.call_process_api(
               ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/Users/nitish/opt/anaconda3/envs/llms/lib/python3.11/site-packages/gradio/route_utils.py", line 322, in call_process_api
    output = await app.get_blocks().process_api(
             ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/Users/nitish/opt/anaconda3/envs/llms/lib/python3.11/site-packages/gradio/blocks.py", line 2113, in process_api
    data = await self.postprocess_data(block_fn, result["prediction"], state)
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/Users/nitish/opt/anaconda3/envs/llms/lib/python3.11/site-packages/gradio/blocks.py", line 1919, in postprocess_data
    prediction_value = block.postprocess(prediction_value)
                       ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^